# Uzyskanie danych o obiektach medycznych w Małopolsce
- Będą one potrzebne w przefiltrowaniu tras pobranych z odczytów GPS, które nie wydają się być trasami karetek

In [2]:
# !pip install osmnx
import osmnx as ox

# Definiujemy słownik z tagami OSM dla obiektów medycznych
tags = {
    'amenity': ['hospital', 'clinic', 'doctors'],
    'emergency': ['ambulance_station']
}

gdf = ox.features_from_place("Małopolskie, Polska", tags)

  Using cached osmnx-2.0.3-py3-none-any.whl (100 kB)
  Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
  Using cached pyogrio-0.11.0-cp39-cp39-win_amd64.whl (19.2 MB)
  Attempting uninstall: geopandas
    Found existing installation: geopandas 0.14.1
    Uninstalling geopandas-0.14.1:
      Successfully uninstalled geopandas-0.14.1


In [3]:
gdf

geometry  \
element id                                                              
node    341400536                           POINT (19.95055 50.06528)   
        342930830                            POINT (19.9438 50.09214)   
        469985535                           POINT (19.94519 50.04752)   
        469988995                           POINT (19.93958 50.05427)   
        475387291                           POINT (19.93466 50.06359)   
...                                                               ...   
way     1348953905  POLYGON ((19.23614 49.98177, 19.23616 49.98173...   
        1357798113  POLYGON ((20.70642 49.62372, 20.70649 49.62379...   
        1358184219  POLYGON ((19.94476 49.93545, 19.94483 49.93544...   
        1379182888  POLYGON ((21.00803 50.01974, 21.00803 50.01976...   
        1383274655  POLYGON ((20.77968 50.24446, 20.77949 50.24444...   

                     amenity  \
element id                     
node    341400536     clinic   
        342930830    doctors   
        469985535   hospital   
        469988995    doctors   
        475387291    doctors   
...                      ...   
way     1348953905    clinic   
        1357798113    clinic   
        1358184219    clinic   
        1379182888    clinic   
        1383274655    clinic   

                                                                 name  \
element id                                                              
node    341400536                   Małopolski Ośrodek Medycyny Pracy   
        342930830                                                 NaN   
        469985535   Szpital Zakonu Bonifratrów Świętego Jana Grandego   
        469988995   Klinika Janeczko - medycyna estetyczna i derma...   
        475387291   Laboratorium Protetyki Stomatologicznej "Top D...   
...                                                               ...   
way     1348953905  Budynek Ośrodka Zdrowia w Grojcu im dr.Tadeusz...   
        1357798113                                Dr Szymańska Clinic   
        1358184219   Niepubliczny Zakład Opieki Zdrowotnej Sp. z o.o.   
        1379182888  Miejska Przychodnia Lekarska nr 5 Sp. z o.o. NZOZ   
        1383274655      Gminny Zakład Opieki Zdrowotnej w Gręboszowie   

                   healthcare  \
element id                      
node    341400536      clinic   
        342930830      doctor   
        469985535    hospital   
        469988995      doctor   
        475387291      doctor   
...                       ...   
way     1348953905     clinic   
        1357798113        NaN   
        1358184219     clinic   
        1379182888     clinic   
        1383274655     clinic   

                                                        opening_hours  \
element id                                                              
node    341400536                                                 NaN   
        342930830                                                 NaN   
        469985535                                                 NaN   
        469988995                                   Mo-Fr 09:00-20:00   
        475387291                                                 NaN   
...                                                               ...   
way     1348953905                                                NaN   
        1357798113  Mo-Sa 09:00-21:00; Jan 26,Apr 13,Apr 27,Jun 29...   
        1358184219                                                NaN   
        1379182888                                                NaN   
        1383274655                                                NaN   

                   emergency note             phone addr:housenumber  \
element id                                                             
node    341400536        NaN  NaN               NaN                1   
        342930830        NaN  NaN               NaN              NaN   
        469985535        NaN  NaN               NaN              NaN   
    

In [4]:
# sprawdz jakie mamy typy geometrii w kolumnie geometry w gdf
gdf['geometry'].apply(lambda x: x.geom_type).value_counts()

Point         597
Polygon       291
LineString      2
Name: geometry, dtype: int64

In [5]:
# dla każdego obiektu ktory nie jest punktem przekonweruj do punktu
gdf['geometry'] = gdf['geometry'].apply(lambda x: x.representative_point() if x.geom_type != 'Point' else x)

In [8]:
# gdf[gdf['emergency'].notna()]
gdf['emergency'].unique()

array([nan, 'no', 'ambulance_station', 'yes'], dtype=object)

In [13]:
mask = gdf['emergency'].notna() & gdf['amenity'].isna()
# mask.mean()
gdf.loc[mask, 'amenity'] = gdf.loc[mask, 'emergency']
gdf[mask]

geometry            amenity  \
element id                                                          
node    1487800000   POINT (19.42866 50.13608)  ambulance_station   
        2436444315   POINT (19.82085 49.97528)  ambulance_station   
        2791518133   POINT (19.74536 50.21508)  ambulance_station   
        3320073589   POINT (19.33206 49.85554)  ambulance_station   
        4247426089    POINT (20.4035 49.72404)  ambulance_station   
        4545323842   POINT (19.80012 50.11677)  ambulance_station   
        5776813848   POINT (20.22172 50.03005)  ambulance_station   
        6154562676   POINT (19.49842 50.31059)  ambulance_station   
        6154566724   POINT (19.46266 50.15849)  ambulance_station   
        6358804333   POINT (19.80344 50.04616)  ambulance_station   
        7860582175   POINT (19.45049 50.07063)  ambulance_station   
        7860591811   POINT (19.43355 49.99435)  ambulance_station   
        7862454276   POINT (19.34247 49.85549)  ambulance_station   
        7862462374   POINT (19.13585 49.96936)  ambulance_station   
        7862468427   POINT (19.68449 49.86954)  ambulance_station   
        7863061307   POINT (19.21181 49.88078)  ambulance_station   
        8959641070   POINT (20.62748 49.74266)  ambulance_station   
        9285342259   POINT (20.70288 49.61698)  ambulance_station   
        9743913306    POINT (19.58215 50.2692)  ambulance_station   
        10249494210   POINT (19.23416 50.0395)  ambulance_station   
        11375705119  POINT (19.77396 50.37357)  ambulance_station   
        11578972088   POINT (19.4435 50.27721)  ambulance_station   
        12328586601  POINT (20.00415 50.01577)  ambulance_station   
        12670016008  POINT (19.23446 50.03977)  ambulance_station   
way     63455038     POINT (19.95135 50.06034)  ambulance_station   
        161844211    POINT (19.93009 50.00731)  ambulance_station   
        229257786    POINT (20.99205 50.01632)  ambulance_station   
        644540200    POINT (20.81575 49.85607)  ambulance_station   
        758807661    POINT (20.99148 50.01605)  ambulance_station   
        1340890253   POINT (20.08287 49.98292)  ambulance_station   

                                                                  name  \
element id                                                               
node    1487800000                                   Oddział Ratunkowy   
        2436444315         Pogotowie Ratunkowe im. Siegfrieda Greinera   
        2791518133                                                 NaN   
        3320073589                                                 NaN   
        4247426089   Kolumna Transportu Sanitarnego Szpitala Powiat...   
        4545323842                                                 NaN   
        5776813848                                                 NaN   
        6154562676                                   ZRM K01-096 Hutki   
        6154566724                                         ZRM K01-100   
        6358804333                                   ZRM w Kryspinowie   
        7860582175                                      ZRM w Babicach   
        7860591811         Zespół Ratownictwa Medycznego K01-116 Zator   
        7862454276                                   ZRM w Andrychowie   
        7862462374     Zespół Ratownictwa Medycznego K01-118 Brzeszcze   
        7862468427                       ZRM w Kalwarii Zebrzydowskiej   
        7863061307          Zespół Ratownictwa Medycznego K01-114 Kęty   
        8959641070   Podstacja Pogotowia Ratunkowego w Łososinie Do...   
        9285342259                        Sądeckie Pogotowie Ratunkowe   
        9743913306       Stacja Pogotowia Ratunkowego K01-043, K01-094   
        10249494210                                                NaN   
        11375705119                                                NaN   
        11578972088                                        ZRM K01-152   
        12328586601  Krakowskie Pogotowie Ratunkowe, 

In [14]:
to_save = gdf[['name', 'geometry', 'amenity']].copy()

In [15]:
to_save[to_save['name'].str.startswith('Powiatowa Stacja', na=False)]

name  \
element id                                                             
way     229257786  Powiatowa Stacja Pogotowia Ratunkowego w Tarnowie   
        758807661  Powiatowa Stacja Pogotowia Ratunkowego w Tarnowie   

                                    geometry            amenity  
element id                                                       
way     229257786  POINT (20.99205 50.01632)  ambulance_station  
        758807661  POINT (20.99148 50.01605)  ambulance_station

In [16]:
# znajdz i wyswietl zduplikowane wartosci kolumny name
duplicates = to_save[to_save.duplicated(['name'], keep=False)]
duplicates = duplicates.sort_values('name')
duplicates

name                   geometry   amenity
element id                                                         
way     715008046    Allmedica  POINT (20.05161 49.47743)   doctors
node    4325736389   Allmedica   POINT (20.0272 49.48161)    clinic
        5749682188   Allmedica  POINT (19.96691 49.30505)    clinic
way     245724231    Allmedica  POINT (19.51196 49.87611)    clinic
node    10936266848  Allmedica  POINT (20.08039 49.67315)    clinic
...                        ...                        ...       ...
way     1245036087         NaN  POINT (19.81497 49.93914)  hospital
        1249039656         NaN  POINT (19.73765 49.94418)  hospital
        1293521768         NaN  POINT (19.69352 49.72872)  hospital
        1314439570         NaN   POINT (20.06487 49.9826)    clinic
        1332799163         NaN  POINT (19.43363 50.16747)    clinic

[248 rows x 3 columns]

In [17]:
# usun te wiersze z wyjsciowego gdf, ktore maja ta sama nazwe oraz są blisko siebie
DIST_THRESHOLD = 300

to_save = to_save.set_geometry('geometry')
to_save = to_save.set_crs(4326).to_crs(3857)

# Find duplicate names
duplicates = to_save[to_save.duplicated(['name'], keep=False)].copy()

# Indices to drop
to_drop = set()

# Group by name and check distances
for name, group in duplicates.groupby('name'):
    coords = list(group.geometry)
    n = len(coords)
    for i in range(n):
        for j in range(i + 1, n):
            if coords[i].distance(coords[j]) < DIST_THRESHOLD:
                to_drop.update(group.index)

# Drop close duplicates from to_save
to_save_cleaned = to_save.drop(index=to_drop)

In [18]:
to_drop

{('node', 1740702612),
 ('node', 2823530338),
 ('node', 3171194975),
 ('node', 3192927501),
 ('node', 3306492514),
 ('node', 3845472623),
 ('node', 5130699125),
 ('node', 5511490552),
 ('node', 5738939736),
 ('node', 6782290553),
 ('node', 7908106202),
 ('node', 11434799002),
 ('way', 229257786),
 ('way', 229425790),
 ('way', 241271412),
 ('way', 391518215),
 ('way', 515275326),
 ('way', 714375036),
 ('way', 758807661),
 ('way', 1379182888)}

In [19]:
to_save_cleaned

name  \
element id                                                              
node    341400536                   Małopolski Ośrodek Medycyny Pracy   
        342930830                                                 NaN   
        469985535   Szpital Zakonu Bonifratrów Świętego Jana Grandego   
        469988995   Klinika Janeczko - medycyna estetyczna i derma...   
        475387291   Laboratorium Protetyki Stomatologicznej "Top D...   
...                                                               ...   
way     1340890253         Krakowskie Pogotowie Ratunkowe w Wieliczce   
        1348953905  Budynek Ośrodka Zdrowia w Grojcu im dr.Tadeusz...   
        1357798113                                Dr Szymańska Clinic   
        1358184219   Niepubliczny Zakład Opieki Zdrowotnej Sp. z o.o.   
        1383274655      Gminny Zakład Opieki Zdrowotnej w Gręboszowie   

                                           geometry            amenity  
element id                                                              
node    341400536   POINT (2220884.677 6457589.184)             clinic  
        342930830   POINT (2220133.616 6462247.385)            doctors  
        469985535   POINT (2220287.994 6454510.007)           hospital  
        469988995   POINT (2219663.592 6455679.499)            doctors  
        475387291   POINT (2219116.211 6457295.488)            doctors  
...                                             ...                ...  
way     1340890253  POINT (2235614.717 6443318.688)  ambulance_station  
        1348953905   POINT (2141380.42 6443116.087)             clinic  
        1357798113  POINT (2305039.213 6381372.994)             clinic  
        1358184219  POINT (2220232.085 6435085.822)             clinic  
        1383274655  POINT (2313174.971 6488705.637)             clinic  

[869 rows x 3 columns]

In [20]:
# zapisz obiekt to_save do pliku csv
to_save_cleaned.to_csv(r"X:\dane_karetki\hospitals.csv", index=False)